# Tutorial 9 - Changing the mesh

在教程8中，我们了解了如何更改解算器选项。在本教程中，我们将更改模拟中使用的网格，并演示如何研究网格对解决方案的影响。

PyBaMM中的所有模型都具有模拟中使用的默认网格点数量。但是，根据模拟的操作条件或使用的参数等情况，您可能会发现需要增加网格中的点数以获得准确的解。另一方面，您可能会发现，您能够减少网格点的数量，并且仍然能够以可接受的精度获得解决方案，但计算时间更短。

进行网格优化研究始终是一种很好的做法，您可以使用更精细的网格模拟相同的问题并比较结果。这里将以图形方式显示如何进行此操作，但在实践中，您可能希望对相对误差进行更详细的计算。

In [1]:
import pybamm

## 更改网格中的点数。

首先，我们加载一个模型

In [2]:
model = pybamm.lithium_ion.SPMe()

然后我们可以查看默认点数，这些点数存储为字典，字典的键是每个域的变量。

In [3]:
model.default_var_pts

{'x_n': 20,
 'x_s': 20,
 'x_p': 20,
 'r_n': 20,
 'r_p': 20,
 'y': 10,
 'z': 10,
 'R_n': 30,
 'R_p': 30}

要使用不同数量的点运行模拟，我们可以定义自己的字典

In [4]:
# get the spatial variables used in pybamm
var = pybamm.standard_spatial_vars  

# create our dictionary 
var_pts = {
    var.x_n: 10,  # negative electrode
    var.x_s: 10,  # separator 
    var.x_p: 10,  # positive electrode
    var.r_n: 10,  # negative particle
    var.r_p: 10,  # positive particle
}

然后，我们创建并解决一个模拟，将点字典作为关键字参数传递

In [5]:
sim = pybamm.Simulation(model, var_pts=var_pts)
sim.solve([0, 3600])

In [6]:
sim.plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

## 进行网格优化研究

为了研究网格对解决方案的影响，我们必须多次求解模型，同时提高网格分辨率。我们首先创建一个列表，列出每个域要使用的点数

In [10]:
npts = [4, 8, 16, 32, 64,128]

现在我们可以在列表上循环，一边创建和求解仿真。解决方案存储在解决方案列表中

In [11]:
#选择型号和参数
model = pybamm.lithium_ion.DFN()
chemistry = pybamm.parameter_sets.Ecker2015
parameter_values = pybamm.ParameterValues(chemistry=chemistry)

#选择解算器
solver = pybamm.CasadiSolver(mode="fast")

#循环覆盖网格点的数量
solutions = []
for N in npts:
    var_pts = {
        var.x_n: N,  # negative electrode
        var.x_s: N,  # separator 
        var.x_p: N,  # positive electrode
        var.r_n: N,  # negative particle
        var.r_p: N,  # positive particle
    }    
    sim = pybamm.Simulation(
        model, solver=solver, parameter_values=parameter_values, var_pts=var_pts
    )
    sim.solve([0, 3600])
    solutions.append(sim.solution)

现在，我们可以将我们的解决方案列表传递给动态绘图方法，从而可以查看网格对计算的终端电压的影响。我们使用labels关键字传递点列表，以便使用模拟中使用的点的数量标记图

In [12]:
pybamm.dynamic_plot(solutions, ["Terminal voltage [V]"], time_unit="seconds", labels=npts)  

interactive(children=(FloatSlider(value=0.0, description='t', max=3600.0, step=36.0), Output()), _dom_classes=…